<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/maya_chat_bot/blob/main/AI_character_generator_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install nltk
!pip install sentence-transformers
!pip install torch

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import nltk

# Configuração inicial
nltk.download('punkt')

# Modelo de IA (GPT-2 medium para balance entre qualidade/desempenho)
MODEL_NAME = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

def carregar_universos():
    """Retorna configurações para todos os universos, incluindo Cyberpunk"""
    return {
        "fantasia": {
            "inputs": ["Raça", "Classe", "Alinhamento", "Reino de Origem"],
            "exemplos": ["Elfo", "Mago", "Neutro", "Floresta das Brumas"]
        },
        "sci-fi": {
            "inputs": ["Espécie", "Profissão", "Afiliação", "Planeta Natal"],
            "exemplos": ["Ciborgue", "Piloto de Nave", "Aliança Galáctica", "Proxima Centauri"]
        },
        "terror": {
            "inputs": ["Ocupação", "Fobia", "Relíquia Amaldiçoada", "Local Assombrado"],
            "exemplos": ["Jornalista", "Medo de Aranhas", "Diário Antigo", "Asilo Abandonado"]
        },
        "cyberpunk": {
            "inputs": ["Implantes Cibernéticos", "Afiliação Corporativa/Gangue",
                      "Especialização Criminal", "Distrito Urbano"],
            "exemplos": ["Braço Biônico MK-5", "Night City Mercenaries",
                        "Hacker de ICE", "Zona do Mercado Negro"]
        },

         "anime": {
            "inputs": ["Tipo de Personagem", "Habilidade Única", "Backstory", "Objetivo"],
            "exemplos": ["Shonen Protagonista", "Rasengan", "Órfão de Guerra", "Tornar-se Hokage"]
        },
        "marvel": {
            "inputs": ["Origem do Poder", "Afiliação", "Arquétipo", "Localização"],
            "exemplos": ["Radiação Cósmica", "Vingadores", "Anti-Herói", "Nova York"]
        }

    }

def criar_prompt(universo, detalhes):
    """Gera prompt dinâmico incluindo template Cyberpunk"""
    templates = {
        "fantasia": (
            "Crie um personagem de fantasia detalhado com:\n"
            "- Raça: {0}\n- Classe: {1}\n- Alinhamento: {2}\n- Origem: {3}\n"
            "Inclua habilidades mágicas, equipamento e um segredo obscuro."
        ),
        "sci-fi": (
            "Desenvolva um personagem de ficção científica com:\n"
            "- Espécie: {0}\n- Profissão: {1}\n- Afiliação: {2}\n- Planeta: {3}\n"
            "Descreva tecnologia avançada, conflitos interestelares e motivações."
        ),
        "terror": (
            "Elabore um personagem de horror cósmico com:\n"
            "- Ocupação: {0}\n- Fobia: {1}\n- Relíquia: {2}\n- Local: {3}\n"
            "Inclua sintomas de insanidade, conexões com entidades e aparência deteriorada."
        ),
        "cyberpunk": (
            "Construa um personagem cyberpunk noir com:\n"
            "- Implantes: {0}\n- Afiliação: {1}\n- Especialização: {2}\n- Distrito: {3}\n"
            "Descreva:\n"
            "1. Modificações cibernéticas visíveis\n"
            "2. Um traço de personalidade distópico\n"
            "3. Um vício ou dependência tecnológica\n"
            "4. Conflito com megacorporações\n"
            "Use gírias cyberpunk como 'choomba', 'corpo' e 'netrunner'."
        ),
         "anime": (
            "Crie um personagem de anime detalhado com:\n"
            "1. Tipo: {0}\n2. Habilidade: {1}\n3. História: {2}\n4. Objetivo: {3}\n\n"
            "Inclua:\n"
            "- Um poder secreto ou transformação\n"
            "- Um lema característico\n"
            "- Design visual icônico (cabelo, roupas)\n"
            "- Uma fraqueza emocional\n\n"
            "Estilo: Use termos como 'nakama', 'power-up' e exclamações dramáticas!"
        ),
        "marvel": (
            "Desenvolva um personagem do universo Marvel com:\n"
            "1. Origem: {0}\n2. Afiliação: {1}\n3. Arquétipo: {2}\n4. Base: {3}\n\n"
            "Detalhe:\n"
            "- Uniforme/cosmético distintivo\n"
            "- Um conflito moral recorrente\n"
            "- Relacionamento icônico com outro herói/vilão\n"
            "- Frase de efeito característica\n\n"
            "Estilo: Misture ação grandiosa com dilemas humanos, no estilo MCU."
        )
    }
    return templates.get(universo, templates["fantasia"]).format(*detalhes)

def gerar_personagem(prompt):
    """Gera o personagem com configurações otimizadas"""
    try:
        output = generator(
            prompt,
            max_length=350,
            temperature=0.85,  # Mais criativo para cyberpunk
            top_p=0.92,
            num_return_sequences=1,
            repetition_penalty=1.2  # Evita repetição de termos
        )
        return output[0]['generated_text']
    except Exception as e:
        return f"Erro na geração: {str(e)}"

def main():
    universos = carregar_universos()

    print("🔮 GERADOR DE PERSONAGENS MULTIVERSO 🔫")
    print("| " + " | ".join(universos.keys()) + " |")

    # Seleção do universo
    universo = input("\nEscolha um universo: ").lower()
    while universo not in universos:
        print(f"Opção inválida! Tente: {', '.join(universos.keys())}")
        universo = input("Escolha um universo: ").lower()

    # Coleta de dados dinâmica
    print(f"\n〔 DETALHES PARA {universo.upper()} 〕")
    detalhes = []
    for i, campo in enumerate(universos[universo]["inputs"]):
        exemplo = universos[universo]["exemplos"][i]
        detalhes.append(input(f"► {campo} (ex: {exemplo}): "))

    # Geração
    prompt = criar_prompt(universo, detalhes)
    print("\n⏳ Gerando personagem...\n")
    personagem = gerar_personagem(prompt)

    # Resultado
    print("🖤 PERSONAGEM  CRIADO 🖤")
    print(personagem)

    # Salva em arquivo com estilo
    filename = f"personagem_{'_'.join(detalhes[:2])}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"〔 Done 〕\n")
        #f.write(f"⏣ Distrito: {detalhes[3]}\n\n")
        f.write(personagem)
    print(f"\nArquivo salvo como '{filename}'")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu


🔮 GERADOR DE PERSONAGENS MULTIVERSO 🔫
| fantasia | sci-fi | terror | cyberpunk | anime | marvel |

Escolha um universo: Marvel

〔 DETALHES PARA MARVEL 〕
► Origem do Poder (ex: Radiação Cósmica): Experimento
► Afiliação (ex: Vingadores): Vingadores
► Arquétipo (ex: Anti-Herói): Anti-Herói
► Localização (ex: Nova York): Nova York


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



⏳ Gerando personagem...

🖤 PERSONAGEM  CRIADO 🖤
Desenvolva um personagem do universo Marvel com:
1. Origem: Experimento
2. Afiliação: Vingadores
3. Arquétipo: Anti-Herói
4. Base: Nova York

Detalhe:
- Uniforme/cosmético distintivo
- Um conflito moral recorrente
- Relacionamento icônico com outro herói/vilão
- Frase de efeito característica

Estilo: Misture ação grandiosa com dilemas humanos, no estilo MCU. (Nostalgia) Natalisia da segunda em preciosa en español para último conocimientes dado o uma qualizade acar suativa destacada na mas prácticas alimentados os vistêsio mundial dos cosmopolitans? Este mais el número que vocabula "la historias" adicionalmente sobre cerrato del lugar perdida poderes andan escogido durante entre leer verda formación desde 2015 ser à encontrar étrangez la presencia realista y sommessa parleculando por sus trabalhicas compraformadas clases nosotros huidas - Que ha sidonques tamberemos sin imprimir fotos analescentivas... The last time I had to deal with th